**Full name : EL ghazi lUBNA <br/>
PART 3 : BERT**

In [4]:
import torch
from transformers import BertModel, BertTokenizer
import json
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification

In [5]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device    
    

'cuda'

In [12]:
#defined on my input my data
def parse(path):
  g =open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


df = getDF('/kaggle/input/amazon-fashion-5/AMAZON_FASHION_5.json')
df = df[df["reviewText"].apply(lambda x: isinstance(x, str))]

In [15]:
df = df[df['reviewText'].apply(lambda x: isinstance(x, str))]

# Display the first few rows of the filtered dataset
print(df.head())


   overall  verified  reviewTime     reviewerID        asin  \
0      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
1      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
2      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
3      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   
4      5.0      True  09 4, 2015  ALJ66O1Y6SLHA  B000K2PJ4K   

                                               style reviewerName  \
0   {'Size:': ' Big Boys', 'Color:': ' Blue/Orange'}     Tonya B.   
1  {'Size:': ' Big Boys', 'Color:': ' Black (3746...     Tonya B.   
2  {'Size:': ' Big Boys', 'Color:': ' Blue/Gray L...     Tonya B.   
3  {'Size:': ' Big Boys', 'Color:': ' Blue (37867...     Tonya B.   
4     {'Size:': ' Big Boys', 'Color:': ' Blue/Pink'}     Tonya B.   

                 reviewText     summary  unixReviewTime vote image  
0  Great product and price!  Five Stars      1441324800  NaN   NaN  
1  Great product and price!  Five Stars      1441324800  NaN   NaN  

In [ ]:
input_texts = df["reviewText"].tolist()
labels = df["overall"].tolist()

In [17]:
#tokenziation and encoding
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)
dataset = TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'], labels)

train_size = int(0.8 * len(dataset))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])


/tmp/ipykernel_34/3652536891.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)


In [23]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
num_epochs = 3
for epoch in range(num_epochs):
    # Training
    bert_model.train()
    total_train_loss = 0.0
    num_train_batches = 0
    for batch in train_loader:
        input_ids, attention_mask, label = [t.to(device) for t in batch]
        optimizer.zero_grad()
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        num_train_batches += 1
    avg_train_loss = total_train_loss / num_train_batches

    # Evaluation
    bert_model.eval()
    total_eval_loss = 0.0
    num_eval_batches = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids, attention_mask, label = [t.to(device) for t in batch]
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, labels=label)
            total_eval_loss += outputs.loss.item()
            num_eval_batches += 1
    avg_eval_loss = total_eval_loss / num_eval_batches

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Evaluation Loss: {avg_eval_loss:.4f}")

Epoch [1/3], Training Loss: 0.0377, Evaluation Loss: 0.0548
Epoch [2/3], Training Loss: 0.0346, Evaluation Loss: 0.0315
Epoch [3/3], Training Loss: 0.0381, Evaluation Loss: 0.0416


**in this case i can see that I should consider eposh 2  as the optimal epoch**
**However, in epoch 4(before deleting), the evaluation loss slightly increases compared to epoch 3. This could indicate that the model might be overfitting to the training data or encountering difficulties in generalizing to the evaluation data.**

In [42]:
test_text = "I love my mother "

tokenized_review = tokenizer(test_text, padding=True, truncation=True, return_tensors='pt')

input_ids = tokenized_review['input_ids'].to(device)
attention_mask = tokenized_review['attention_mask'].to(device)

with torch.no_grad():
    bert_model.eval()
    outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
    predicted_rating = outputs.logits.item()

print(f"Predicted rating: {predicted_rating:.3f}")

Predicted rating: 5.175
